# How to create an Agent


This tries to setup the example at https://learn.microsoft.com/en-us/azure/ai-services/agents/quickstart?pivots=programming-language-python-azure
    

## Set Up
The following libraries must be installed to use LangChain with Azure OpenAI.<br>

In [1]:
# INSTALLATION
%pip install openai
%pip install langchain
%pip install langchain-openai

# this is only for loading images, not for the ai part
%pip install imageio

%pip install azure-ai-projects
%pip install azure-identity


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pi

## Agent creation




In [2]:
import os
import json

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path

In [3]:

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base=config_details['OPENAI_API_BASE']
    
# API version e.g. "2023-07-01-preview"
openai_api_version=config_details['OPENAI_API_VERSION']

# The name of your Azure OpenAI deployment chat model. e.g. "gpt-35-turbo-0613"
deployment_name=config_details['DEPLOYMENT_NAME']

# The API key for your Azure OpenAI resource.
openai_api_key = config_details['OPENAI_API_KEY']
# openai_api_key = os.getenv("OPENAI_API_KEY")

# This is set to `azure`
openai_api_type="azure"

# This is to setup the foundry project connection string
foundry_connection_string=config_details['PROJECT_CONNECTION_STRING']

In [4]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# HostName can be found by navigating to your discovery_url and removing the leading "https://" and trailing "/discovery"
# To find your discovery_url, run the CLI command: az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url
# Project Connection example: eastus.api.azureml.ms;12345678-abcd-1234-9fc6-62780b3d3e05;my-resource-group;my-project-name
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=foundry_connection_string
)

This is is just an issue:

HttpResponseError: (model_not_found_or_ready) The model deployment provided for this resource does not exist or is not yet ready. If you created the deployment within the last 5 minutes, please wait a moment and try again.
Code: model_not_found_or_ready
Message: The model deployment provided for this resource does not exist or is not yet ready. If you created the deployment within the last 5 minutes, please wait a moment and try again.

In [5]:
with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()
    
    # The CodeInterpreterTool needs to be included in creation of the agent
    agent = project_client.agents.create_agent(
        model=deployment_name,
        name="my-first-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    
    
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
    )
    print(f"Created message, message ID: {message.id}")
    
    # Run the agent
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Get the last message from the sender
    last_msg = messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # Generate an image file for the bar chart
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")

    # Print the file path(s) from the messages
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")
        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


Created agent, agent ID: asst_bo4Dlt6rAETfFmFbviHGvzim
Created thread, thread ID: thread_rFpbQq5XAAf08uUpSDPin6MI
Created message, message ID: msg_kE2s07Aiij4aSRtLwr3oUpS5
Run finished with status: RunStatus.FAILED
Run failed: {'code': 'rate_limit_exceeded', 'message': 'Rate limit is exceeded. Try again in 86400 seconds.'}
Messages: {'object': 'list', 'data': [{'id': 'msg_kE2s07Aiij4aSRtLwr3oUpS5', 'object': 'thread.message', 'created_at': 1738227567, 'assistant_id': None, 'thread_id': 'thread_rFpbQq5XAAf08uUpSDPin6MI', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_kE2s07Aiij4aSRtLwr3oUpS5', 'last_id': 'msg_kE2s07Aiij4aSRtLwr3oUpS5', 'has_more': False}
Deleted agent
